In [12]:
#python SDK used to acces gemini Models
!pip install -q -U google-generativeai
     

import google.generativeai as genai

## Extracting Invoice Information from Documents

In [13]:
#loading api_key sercurely from .env
import os
from dotenv import load_dotenv

In [14]:
#load .env file
load_dotenv()

#Read api key from .env
api_key = os.getenv("gemini_api")

In [15]:
genai.configure(api_key=api_key)

In [16]:
#list of gemini model that support content generation 
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash

In [17]:

# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [18]:
#initialize the gemini model
model = genai.GenerativeModel(
    model_name="models/gemini-2.5-pro",
    generation_config=MODEL_CONFIG,
    safety_settings=safety_settings
)

     

In [19]:
#Loads the image/pdf from disk and packages it for the Gemini API.
from pathlib import Path
import mimetypes

def image_format(file_path):
    file = Path(file_path)
    if not file.exists():
        raise FileNotFoundError(f"Could not find file: {file}")

    mime_type, _ = mimetypes.guess_type(file)
    if mime_type not in ["image/png", "image/jpeg", "application/pdf"]:
        raise ValueError(f"Unsupported file type: {mime_type}")

    return [{
        "mime_type": mime_type,
        "data": file.read_bytes()
    }]

In [20]:
#Sends the image + prompt to Gemini and returns the response text.
def gemini_output(file_path, system_prompt, user_prompt):
    file_info = image_format(file_path)
    input_prompt = [system_prompt, file_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text


In [21]:
#You give context and instructions via prompt and send the image path.
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
file_path = "./Template/sample-invoice.pdf"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required"
     

In [22]:
output = gemini_output(file_path, system_prompt, user_prompt)

### PDF Invoice Output

In [23]:
#Renders the output as Markdown in Jupyter Notebook — helpful for readability.
from IPython.display import Markdown
Markdown(output)

```json
{
  "issuer": {
    "name": "CPB SOFTWARE (GERMANY) GMBH",
    "address": "Im Bruch 3, 63897 Miltenberg",
    "phone": "+49 9371 9786 0",
    "email": "germany@cpb-software.com",
    "website": "www.cpb-software.com",
    "vat_number": "DE199378386"
  },
  "customer": {
    "name": "Musterkunde AG",
    "attention": "Mr. John Doe",
    "address": "Musterstr. 23, 12345 Musterstadt",
    "customer_id": "12345"
  },
  "invoice_id": "123100401",
  "invoice_date": "2024-03-01",
  "billing_period": {
    "start_date": "2024-02-01",
    "end_date": "2024-02-29"
  },
  "currency": "EUR",
  "line_items": [
    {
      "description": "Basic Fee wmView",
      "quantity": 1,
      "unit_price": 130.00,
      "total_amount": 130.00
    },
    {
      "description": "Basis fee for additional user accounts",
      "quantity": 0,
      "unit_price": 10.00,
      "total_amount": 0.00
    },
    {
      "description": "Basic Fee wmPos",
      "quantity": 0,
      "unit_price": 50.00,
      "total_amount": 0.00
    },
    {
      "description": "Basic Fee wmGuide",
      "quantity": 0,
      "unit_price": 1000.00,
      "total_amount": 0.00
    },
    {
      "description": "Change of user accounts",
      "quantity": 0,
      "unit_price": 10.00,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee T1",
      "quantity": 14,
      "unit_price": 0.58,
      "total_amount": 8.12
    },
    {
      "description": "Transaction Fee T2",
      "quantity": 0,
      "unit_price": 0.70,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee T3",
      "quantity": 162,
      "unit_price": 1.50,
      "total_amount": 243.00
    },
    {
      "description": "Transaction Fee T4",
      "quantity": 0,
      "unit_price": 0.50,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee T5",
      "quantity": 0,
      "unit_price": 0.80,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee T6",
      "quantity": 0,
      "unit_price": 1.80,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G1",
      "quantity": 0,
      "unit_price": 0.30,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G2",
      "quantity": 0,
      "unit_price": 0.30,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G3",
      "quantity": 0,
      "unit_price": 0.40,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G4",
      "quantity": 0,
      "unit_price": 0.40,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G5",
      "quantity": 0,
      "unit_price": 0.30,
      "total_amount": 0.00
    },
    {
      "description": "Transaction Fee G6",
      "quantity": 0,
      "unit_price": 0.30,
      "total_amount": 0.00
    }
  ],
  "subtotal": 381.12,
  "tax_details": [
    {
      "description": "VAT 19%",
      "amount": 72.41
    }
  ],
  "total": 453.53,
  "payment_terms": "Immediate payment without discount. Any bank charges must be paid by the invoice recipient. Bank fees at our expense will be charged to the invoice recipient!",
  "payment_details": {
    "iban": "DE29 1234 5678 9012 3456 78",
    "bic": "GENODE51MIC",
    "transfer_type": "SEPA Credit Transfer"
  },
  "notes": "This invoice is generated automatically and will not be signed",
  "invoice_details": {
    "user_account_breakdown": [
      {
        "account": "user-account-1",
        "queries": [
          {
            "type": "T1",
            "count": 10
          },
          {
            "type": "T3",
            "count": 99
          }
        ],
        "total_cost": 154.30
      },
      {
        "account": "user-account-2",
        "queries": [
          {
            "type": "T1",
            "count": 4
          },
          {
            "type": "T3",
            "count": 63
          }
        ],
        "total_cost": 96.82
      }
    ],
    "query_reference_breakdown": [
      {
        "reference": "*Not specified*",
        "queries": [
          {
            "type": "T1",
            "count": 4
          },
          {
            "type": "T3",
            "count": 9
          }
        ],
        "total_cost": 15.82
      },
      {
        "reference": "AZR/31/27439",
        "queries": [
          {
            "type": "T3",
            "count": 12
          }
        ],
        "total_cost": 18.00
      },
      {
        "reference": "CCL/3715",
        "queries": [
          {
            "type": "T3",
            "count": 4
          }
        ],
        "total_cost": 6.00
      },
      {
        "reference": "CRS/28432",
        "queries": [
          {
            "type": "T1",
            "count": 5
          },
          {
            "type": "T3",
            "count": 36
          }
        ],
        "total_cost": 56.90
      },
      {
        "reference": "Cs/52113",
        "queries": [
          {
            "type": "T3",
            "count": 19
          }
        ],
        "total_cost": 28.50
      },
      {
        "reference": "GS 32090

### Invoice Image Output

In [24]:
file_path = "./Invoice/caption.jpg"
output_pdf = gemini_output(file_path, system_prompt, user_prompt)
Markdown(output_pdf)

```json
{
  "merchant_name": "PALMİYE RESTAURANT CAFE BAR",
  "merchant_address": "İncili Çavuş Sok. No: 35 Sultanahmet / İSTANBUL",
  "merchant_phone": "(0212) 527 64 76",
  "transaction_date": "17/08/2024",
  "invoice_number": "115059",
  "waiter": "Murat",
  "line_items": [
    {
      "description": "Q/O",
      "amount": 390.00
    },
    {
      "description": "Bolovz",
      "amount": 390.00
    },
    {
      "description": "King",
      "amount": 2000.00
    },
    {
      "description": "Börek - 1",
      "amount": 250.00
    },
    {
      "description": "Sprat XX",
      "amount": 480.00
    }
  ],
  "total": 3650.00
}
```